# Geocoding

Geocoding is the process of getting coordinates (latitude, longitude) for a street address or landmark.  You can also do the reverse and get a street address for a given set of coordinates.  In order to accomplish this task you need an up to date system of record to map back and forth which is something the [HERE Geocoder API](https://developer.here.com/documentation/geocoder/topics/what-is.html) can provide.

We'll look at a few common libraries that can be used for geocoding operations:
* `requests`
* `geopy`
* `geocoder`
* `geopandas`

After looking at a few more variations you can experiment with a few example projects.

* Analyzing Image Metadata
* Analyzing Text

# Credentials

Access control for HERE services uses an APP_ID and APP_CODE that can be retrieved from the [Developer Portal](https://developer.here.com/projects) for your project.  I like to store these as environment variables.

In [ ]:
import os

APP_ID_HERE = os.environ['APP_ID_HERE']
APP_CODE_HERE = os.environ['APP_CODE_HERE']

In [ ]:
TEST_ADDRESS = '300 Lakeside Ave. Cleveland, OH 44113'

# Popular Python Libraries for Geocoding

## Requests

Simple example of making an HTTP call with the **requests** library to retrieve a street address.

In [ ]:
import json
import requests

uri = 'https://geocoder.api.here.com/6.2/geocode.json'
params = {
    'app_id': APP_ID_HERE,
    'app_code': APP_CODE_HERE,
    'searchtext': TEST_ADDRESS,
}

response = requests.get(uri, params=params)
data = json.loads(response.text)
print(json.dumps(data, indent=4, sort_keys=True))

## Geopy

A client to consider instead of direct `requests` is `geopy`.  It includes classes for multiple geocoder services including HERE.

https://pypi.org/project/geopy/

In [ ]:
from geopy.geocoders import Here

geocoder = Here(APP_ID_HERE, APP_CODE_HERE)
result = geocoder.geocode(TEST_ADDRESS)
result.address

In [ ]:
result.raw

## Geocoder

Another library to consider is `geocoder`.

https://geocoder.readthedocs.io/

In [ ]:
import geocoder

result = geocoder.here(TEST_ADDRESS, app_id=APP_ID_HERE, app_code=APP_CODE_HERE)
result.latlng

## GeoPandas

GeoPandas is another powerful library that provides geocoding methods.

http://geopandas.org/geocoding.html

In [ ]:
import geopandas

result = geopandas.tools.geocode(TEST_ADDRESS, provider='Here', app_id=APP_ID_HERE, app_code=APP_CODE_HERE)
result

# More Examples

Depending on your use case the underlying Geocoding APIs from HERE can give much finer level control over the results and how you might use it.

In [ ]:
GEOCODER_URI = 'https://geocoder.api.here.com/6.2/geocode.json'
REVERSE_GEOCODER_URI = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json'

## Bounding Box

When displaying a map you may already have a viewport that allows you to search for partial matches within that area.

In [ ]:
params = {
    'app_id': APP_ID_HERE,
    'app_code': APP_CODE_HERE,
    'mapview': '42.3902,-71.1293;42.3312,-71.0228',
    'searchtext': '1 main'
}

response = requests.get(GEOCODER_URI, params=params)
response.json()

## Address By Parts

You may know specific parts of the street address and want to do some marginal address validation.

In [ ]:
params = {
    'app_id': APP_ID_HERE,
    'app_code': APP_CODE_HERE,
    'housenumber': '427',
    'street': 'randolph',
    'city': 'chicago'
}

response = requests.get(GEOCODER_URI, params=params)
response.json()

## Street Intersection

You may not be looking for a specific building footprint but rather a roadway or intersection.

In [ ]:
params = {
    'app_id': APP_ID_HERE,
    'app_code': APP_CODE_HERE,
    'street': 'invaliden @ chaussee',
    'city': 'berlin',
    'country': 'germany'
}

response = requests.get(GEOCODER_URI, params=params)
response.json()

## Reverse Geocode

HERE provides a positioning API that can be used to find an approximate latitude and longitude based on GPS, Cell Towers, and WiFi access points as part of a radio map database.

If you have a lat, long you can reverse geocode to identify a street address.

In [ ]:
params = {
    'app_id': APP_ID_HERE,
    'app_code': APP_CODE_HERE,
    'prox': '41.504,-81.6941,250',
    'mode': 'retrieveAddresses',
    'maxresults': '1'
}

response = requests.get(REVERSE_GEOCODER_URI, params=params)
response.json()['Response']['View'][0]['Result'][0]['Location']

## Shape Geometry

The geocoder can also retrieve geometry representing the border of a particular area such as a zip code.  See Python below that you can work with or that this webapp also demonstrates.

https://dbabbs.github.io/geojson-shape-tool/

In [ ]:
import shapely.wkt

params = {
    'app_id': APP_ID_HERE,
    'app_code': APP_CODE_HERE,
    'prox': '41.504,-81.6941,250',
    'mode': 'retrieveAddresses',
    'maxresults': '1',
    'additionaldata': 'IncludeShapeLevel,postalCode'
}

response = requests.get(REVERSE_GEOCODER_URI, params=params)
shp = response.json()['Response']['View'][0]['Result'][0]['Location']['Shape']['Value']
shapely.wkt.loads(shp)